"""
Updated on Wednesday April 02 2025

Purpose: Create aggregated data frame for seasons and time period using early season as the reference period.
@author: Siddharth Chaudhary, Chinmay Deval
"""

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [35]:
# Specify the path to the extracted Parquet file
parquet_file = "/Users/cdeval/Library/CloudStorage/OneDrive-NASA/workspace/hydroseesaw_04_02_2025/Results/grouped_median_discharge.parquet"

# Read the Parquet file into a DataFrame
df = pd.read_parquet(parquet_file)


In [36]:
df.head()
#df.shape

,time,station_name,scenario,discharge
0,1976-01-07,1104150.0,hist,83.478806
1,1976-01-07,1104200.0,hist,0.680449
2,1976-01-07,1104300.0,hist,4.381658
3,1976-01-07,1104450.0,hist,NaN
4,1976-01-07,1104480.0,hist,0.244086


In [37]:
# Convert 'time' column to datetime format
df['time'] = pd.to_datetime(df['time'])

# Extract year, month, and day into separate columns
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day

# Create 'time_period' column based on the year
conditions = [
    (df['year'] < 2006),
    (df['year'] >= 2006) & (df['year'] < 2040),
    (df['year'] >= 2040) & (df['year'] < 2070),
    (df['year'] >= 2070)
]
choices = ['historical', 'early', 'mid', 'end']

df['time_period'] = pd.cut(df['year'], bins=[-float('inf'), 2005, 2039, 2069, float('inf')], labels=choices, right=False)


In [38]:
df = df.dropna()
df.shape

(120614936, 8)

In [39]:
conditions = [
    (df['month'].isin([1, 2, 3])),
    (df['month'].isin([4, 5, 6])),
    (df['month'].isin([7, 8, 9])),
    (df['month'].isin([10, 11, 12]))
]

choices = ['Season 1', 'Season 2', 'Season 3', 'Season 4']

# Create new column 'season'
df['season'] = np.select(conditions, choices)

In [40]:
df.columns

Index(['time', 'station_name', 'scenario', 'discharge', 'year', 'month', 'day',
       'time_period', 'season'],
      dtype='object')

In [41]:
unique_scenario = df['scenario'].unique()

# Print unique values
print(unique_scenario)

['hist' 'rcp4p5' 'rcp8p5']


In [42]:
unique_time_period = df['time_period'].unique()

# Print unique values
print(unique_time_period)

['historical', 'early', 'mid', 'end']
Categories (4, object): ['historical' < 'early' < 'mid' < 'end']


In [43]:
df['time'] = pd.to_datetime(df['time'])

# Filter the DataFrame where scenario is 'early'
filtered_df_rcp4p5 = df[(df['time_period'] == 'early') & (df['scenario'] == 'rcp4p5')]

print(filtered_df_rcp4p5)


               time  station_name scenario  discharge  year  month  day  \
16755960 2006-01-07     1104150.0   rcp4p5  65.950020  2006      1    7   
16755962 2006-01-07     1104200.0   rcp4p5   3.584657  2006      1    7   
16755964 2006-01-07     1104300.0   rcp4p5   1.388593  2006      1    7   
16755968 2006-01-07     1104480.0   rcp4p5   0.000000  2006      1    7   
16755970 2006-01-07     1104500.0   rcp4p5   1.369061  2006      1    7   
...             ...           ...      ...        ...   ...    ...  ...   
53619056 2038-12-30     6984800.0   rcp4p5   0.123562  2038     12   30   
53619058 2038-12-30     6986100.0   rcp4p5   1.980656  2038     12   30   
53619060 2038-12-30     6987050.0   rcp4p5  13.569571  2038     12   30   
53619062 2038-12-30     6987100.0   rcp4p5  35.874016  2038     12   30   
53619064 2038-12-30     6987150.0   rcp4p5   5.896697  2038     12   30   

         time_period    season  
16755960       early  Season 1  
16755962       early  Season 1  


In [44]:

# Define the percentiles to calculate
percentiles = [1, 5, 10, 90, 95, 99]


print(filtered_df_rcp4p5.shape)  # Check if there are any rows


print(df['time_period'].unique())
print(df['scenario'].unique())

percentile_df = filtered_df_rcp4p5.groupby('station_name')['discharge'].quantile([p/100 for p in percentiles]).unstack()


(18256524, 9)
['historical', 'early', 'mid', 'end']
Categories (4, object): ['historical' < 'early' < 'mid' < 'end']
['hist' 'rcp4p5' 'rcp8p5']


In [61]:

# Rename the columns for clarity
percentile_df.columns = [f'p{int(p)}' for p in percentiles]

# Merge the percentiles back into the original DataFrame
filtered_df.groupby('station_name')['discharge']
print(percentile_df)

percentile_df.to_csv('/Users/cdeval/Library/CloudStorage/OneDrive-NASA/workspace/hydroseesaw_04_02_2025/Results/percentile_thresholds_at_stations_with_early_as_baseline.csv')

                    p1        p5       p10         p90         p95         p99
station_name                                                                  
1104150.0     0.911356  3.016048  5.413468  181.815979  226.945702  374.284752
1104200.0     0.011176  0.045931  0.055648    1.419360    1.875435    2.842895
1104300.0     0.000024  0.000170  0.000526    1.782947    2.344092    3.148132
1104480.0     0.000000  0.000000  0.000000    0.000000    0.000000    0.000000
1104500.0     0.022643  0.030721  0.051891    1.008466    1.216985    1.524037
...                ...       ...       ...         ...         ...         ...
6984800.0     0.072631  0.120436  0.121504   82.968037  121.276436  172.321814
6986100.0     0.787956  2.006996  3.285601   18.656515   23.178034   28.426647
6987050.0     1.905736  2.274863  2.545876   12.139941   14.278536   18.976341
6987100.0     5.699624  7.923030  9.601435   36.743090   42.557071   57.162220
6987150.0     1.091009  1.477573  1.799672    6.4033

In [49]:
# remove historical ts from df before merging

df = df[(df['scenario'] != 'hist') & (df['time_period'] != 'historical')]
print(df['time_period'].unique())
print(df)

['early', 'mid', 'end']
Categories (4, object): ['historical' < 'early' < 'mid' < 'end']
                time  station_name scenario  discharge  year  month  day  \
16755960  2006-01-07     1104150.0   rcp4p5  65.950020  2006      1    7   
16755961  2006-01-07     1104150.0   rcp8p5  65.625732  2006      1    7   
16755962  2006-01-07     1104200.0   rcp4p5   3.584657  2006      1    7   
16755963  2006-01-07     1104200.0   rcp8p5   3.097179  2006      1    7   
16755964  2006-01-07     1104300.0   rcp4p5   1.388593  2006      1    7   
...              ...           ...      ...        ...   ...    ...  ...   
121759965 2099-12-30     6987050.0   rcp8p5   7.912493  2099     12   30   
121759966 2099-12-30     6987100.0   rcp4p5  50.412224  2099     12   30   
121759967 2099-12-30     6987100.0   rcp8p5  26.211447  2099     12   30   
121759968 2099-12-30     6987150.0   rcp4p5   8.691892  2099     12   30   
121759969 2099-12-30     6987150.0   rcp8p5   4.085582  2099     12   30   

In [50]:
df = df.merge(percentile_df, on='station_name', how='left')

In [51]:
df.head

<bound method NDFrame.head of                 time  station_name scenario  discharge  year  month  day  \
0         2006-01-07     1104150.0   rcp4p5  65.950020  2006      1    7   
1         2006-01-07     1104150.0   rcp8p5  65.625732  2006      1    7   
2         2006-01-07     1104200.0   rcp4p5   3.584657  2006      1    7   
3         2006-01-07     1104200.0   rcp8p5   3.097179  2006      1    7   
4         2006-01-07     1104300.0   rcp4p5   1.388593  2006      1    7   
...              ...           ...      ...        ...   ...    ...  ...   
104006859 2099-12-30     6987050.0   rcp8p5   7.912493  2099     12   30   
104006860 2099-12-30     6987100.0   rcp4p5  50.412224  2099     12   30   
104006861 2099-12-30     6987100.0   rcp8p5  26.211447  2099     12   30   
104006862 2099-12-30     6987150.0   rcp4p5   8.691892  2099     12   30   
104006863 2099-12-30     6987150.0   rcp8p5   4.085582  2099     12   30   

          time_period    season        p1        p5      

In [52]:
# Create new columns based on the comparison criteria
df['p1_flag'] = df['discharge'] < df['p1']
df['p5_flag'] = df['discharge'] < df['p5']
df['p10_flag'] = df['discharge'] < df['p10']
df['p90_flag'] = df['discharge'] > df['p90']
df['p95_flag'] = df['discharge'] > df['p95']
df['p99_flag'] = df['discharge'] > df['p99']

# Convert boolean flags to integers (1 or 0)
df['p1_flag'] = df['p1_flag'].astype(int)
df['p5_flag'] = df['p5_flag'].astype(int)
df['p10_flag'] = df['p10_flag'].astype(int)
df['p90_flag'] = df['p90_flag'].astype(int)
df['p95_flag'] = df['p95_flag'].astype(int)
df['p99_flag'] = df['p99_flag'].astype(int)

# Display the resulting DataFrame
print(df)

                time  station_name scenario  discharge  year  month  day  \
0         2006-01-07     1104150.0   rcp4p5  65.950020  2006      1    7   
1         2006-01-07     1104150.0   rcp8p5  65.625732  2006      1    7   
2         2006-01-07     1104200.0   rcp4p5   3.584657  2006      1    7   
3         2006-01-07     1104200.0   rcp8p5   3.097179  2006      1    7   
4         2006-01-07     1104300.0   rcp4p5   1.388593  2006      1    7   
...              ...           ...      ...        ...   ...    ...  ...   
104006859 2099-12-30     6987050.0   rcp8p5   7.912493  2099     12   30   
104006860 2099-12-30     6987100.0   rcp4p5  50.412224  2099     12   30   
104006861 2099-12-30     6987100.0   rcp8p5  26.211447  2099     12   30   
104006862 2099-12-30     6987150.0   rcp4p5   8.691892  2099     12   30   
104006863 2099-12-30     6987150.0   rcp8p5   4.085582  2099     12   30   

          time_period    season        p1  ...       p10         p90  \
0              

In [14]:
#df.to_csv('/Users/sidchaudhary/Documents/GitHub/Hydro-Seesaw/Results/station_weekly_flag_counts_early_mid_end.csv')

In [53]:
df_early_mid_end = df[(df['time_period'] == 'early') | (df['time_period'] == 'mid')| (df['time_period'] == 'end')]
flag_counts = df_early_mid_end.groupby(['station_name','scenario','time_period','season'])[['p1_flag', 'p5_flag', 'p10_flag', 'p90_flag', 'p95_flag', 'p99_flag']].sum()
print(flag_counts)

/var/folders/61/rsv0_yrn6zj37sg5hjgdt3q00000gp/T/ipykernel_47629/3653648116.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  flag_counts = df_early_mid_end.groupby(['station_name','scenario','time_period','season'])[['p1_flag', 'p5_flag', 'p10_flag', 'p90_flag', 'p95_flag', 'p99_flag']].sum()


                                            p1_flag  p5_flag  p10_flag  \
station_name scenario time_period season                                 
1104150.0    rcp4p5   historical  Season 1        0        0         0   
                                  Season 2        0        0         0   
                                  Season 3        0        0         0   
                                  Season 4        0        0         0   
                      early       Season 1        0        0         0   
...                                             ...      ...       ...   
6987150.0    rcp8p5   mid         Season 4       10       39        63   
                      end         Season 1        0        0         2   
                                  Season 2        0        2         9   
                                  Season 3       50      144       227   
                                  Season 4       13       42        66   

                                     

In [54]:
annual_sums = flag_counts.groupby(['station_name', 'scenario', 'time_period']).sum()

# Create a DataFrame for the annual sums
annual_sums = annual_sums.reset_index()
annual_sums['season'] = 'Annual'

# Set the multi-index for annual_sums to match flag_counts
annual_sums.set_index(['station_name', 'scenario', 'time_period', 'season'], inplace=True)

# Concatenate the annual sums into the original flag_counts DataFrame
flag_counts = pd.concat([flag_counts, annual_sums])

# Sort the DataFrame if needed
flag_counts.sort_index(inplace=True)

# Print the updated DataFrame
print(flag_counts)

                                            p1_flag  p5_flag  p10_flag  \
station_name scenario time_period season                                 
1104150.0    rcp4p5   historical  Annual          0        0         0   
                                  Season 1        0        0         0   
                                  Season 2        0        0         0   
                                  Season 3        0        0         0   
                                  Season 4        0        0         0   
...                                             ...      ...       ...   
6987150.0    rcp8p5   end         Annual         63      188       304   
                                  Season 1        0        0         2   
                                  Season 2        0        2         9   
                                  Season 3       50      144       227   
                                  Season 4       13       42        66   

                                     

/var/folders/61/rsv0_yrn6zj37sg5hjgdt3q00000gp/T/ipykernel_47629/2943227123.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  annual_sums = flag_counts.groupby(['station_name', 'scenario', 'time_period']).sum()


In [55]:
flag_counts.columns

Index(['p1_flag', 'p5_flag', 'p10_flag', 'p90_flag', 'p95_flag', 'p99_flag'], dtype='object')

In [56]:
flag_counts.to_csv('/Users/cdeval/Library/CloudStorage/OneDrive-NASA/workspace/hydroseesaw_04_02_2025/Results/flag_counts_mid_end_season_with_early_baseline.csv')
#flag_counts_mid.to_parquet('file_name.parquet', engine='pyarrow')

In [57]:
df_flag_counts = pd.read_csv('/Users/cdeval/Library/CloudStorage/OneDrive-NASA/workspace/hydroseesaw_04_02_2025/Results/flag_counts_mid_end_season_with_early_baseline.csv')
df_station_name = pd.read_csv('/Users/cdeval/Library/CloudStorage/OneDrive-NASA/workspace/hydroseesaw_04_02_2025/data/staion_id.csv')
merged_df = pd.merge(df_flag_counts, df_station_name, on='station_name', how='inner')

In [59]:
merged_df.to_csv('/Users/cdeval/Library/CloudStorage/OneDrive-NASA/workspace/hydroseesaw_04_02_2025/Results/lat_lon_flag_counts_mid_end_season_with_early_baseline.csv')